In [1]:
from kafka import KafkaProducer
import json
import websocket
from config import KAFKA_BOOTSTRAP_SERVERS, KAFKA_TOPIC, FINNHUB_API_KEY
 

symbols = ['AAPL','GOOGL','META','MSFT','AMZN','TSLA','NVDA','AMD','ORCL','NFLX','SPY','QQQ']
topic = KAFKA_TOPIC
API_KEY = FINNHUB_API_KEY

producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer= lambda v:json.dumps(v).encode('utf-8')
)

In [2]:
def on_message(ws, message):
    msg = json.loads(message)
    if msg.get("type") == 'trade':
        for trade in msg.get('data'):
            formatted_trade = {
                "p": trade.get("p"),  # price
                "s": trade.get("s"),  # symbol
                "v": trade.get("v"),  # volume
                "t": trade.get("t")   # timestamp (ms)
            }

            producer.send(topic, formatted_trade)
            print(formatted_trade)
            
def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###", close_status_code, close_msg)

def on_open(ws):
    for s in symbols: 
        ws.send(f'{{"type":"subscribe","symbol":"{s}"}}')


websocket.enableTrace(False)
ws = websocket.WebSocketApp(
    f"wss://ws.finnhub.io?token={API_KEY}",
    on_message = on_message,
    on_error = on_error,
    on_close = on_close
)

ws.on_open = on_open
ws.run_forever()

{'p': 90.54, 's': 'NFLX', 'v': 13, 't': 1767742622963}
{'p': 90.54, 's': 'NFLX', 'v': 11, 't': 1767742622964}
{'p': 90.53, 's': 'NFLX', 'v': 20, 't': 1767742633084}
{'p': 623.6, 's': 'QQQ', 'v': 100, 't': 1767742762868}
{'p': 623.61, 's': 'QQQ', 'v': 200, 't': 1767743144443}
{'p': 434.59, 's': 'TSLA', 'v': 50, 't': 1767743192907}
{'p': 90.55, 's': 'NFLX', 'v': 60, 't': 1767743669456}
{'p': 90.54, 's': 'NFLX', 'v': 10, 't': 1767743721509}
{'p': 90.5, 's': 'NFLX', 'v': 20, 't': 1767743724823}
{'p': 90.5, 's': 'NFLX', 'v': 50, 't': 1767743724823}
{'p': 90.5, 's': 'NFLX', 'v': 10, 't': 1767743724823}
{'p': 434.85, 's': 'TSLA', 'v': 40, 't': 1767743732526}
{'p': 434.85, 's': 'TSLA', 'v': 92, 't': 1767743742083}
{'p': 434.92, 's': 'TSLA', 'v': 40, 't': 1767743757381}
{'p': 434.97, 's': 'TSLA', 'v': 125, 't': 1767743760679}
{'p': 90.54, 's': 'NFLX', 'v': 20, 't': 1767743797710}
{'p': 90.54, 's': 'NFLX', 'v': 60, 't': 1767743797711}
{'p': 90.54, 's': 'NFLX', 'v': 130, 't': 1767743866831}
{'p':

True